In [1]:
import cv2
import mediapipe as mp

#Inicialización de MediaPipe con sus utilidades para dibujo
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

mp_pose = mp.solutions.pose
mp_hands= mp.solutions.hands

pose = mp_pose.Pose()
hands =mp_hands.Hands()

#Inicialización de captura de video
cap = cv2.VideoCapture(0)

#Configuración del modelo Holistic
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:

 while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignorando el frame vacio de la camara.")
      continue
  
    pose_r= pose.process(image)
    hands_r=hands.process(image)
    
    #Se prepara la imagen para el procesamiento
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Conversión de imagen a BGR para su muestreo
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    #Detección de rostro
    if results.face_landmarks:
      mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec = None,
        connection_drawing_spec = mp_drawing_styles
        .get_default_face_mesh_contours_style())
      
    
    #Detección y dibujo del cuerpo (pose)
    if pose_r.pose_landmarks:
      mp_drawing.draw_landmarks(
        image,
        pose_r.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec = mp_drawing_styles
        .get_default_pose_landmarks_style())
      

    #Detección y dibujo de las manos
    if hands_r.multi_hand_landmarks:
        for hand_landmarks in hands_r.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))
            
            combined_landmarks= pose_r.pose_landmarks.copy()
            
            hand_landmarks_indices_to_exclude = range(0,5)
            
            for i, landmark in hand_landmarks_indices_to_exclude:
                if i not in hand_landmarks_indices_to_exclude:
                    combined_landmarks.append(landmark)
            
            mp_drawing.draw_landmarks(
            image,
            combined_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))
            cv2.imshow('X', combined_landmarks)
            
    # Muestreo de camara con modelos ya cargados
    cv2.imshow('MediaPipe Holistic y Hand Tracker', image)

    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

d:\UASLP -Semestre 11\TRADUCTOR\S4L\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


AttributeError: copy